In [4]:
sys.path

ModuleNotFoundError: No module named 'sparksql'

In [5]:
# read in data dictionary. NOTE: I changed header manually.
dd = spark.read.option("header", True).csv("/home/iceberg/app/layer/template/PUMS_data_dictionary.csv")
dd.show()

+---------+--------+--------+-------+--------------------+-------------+--------------------+
|heirarchy|   field|category|subtype|           name_text|         code|                text|
+---------+--------+--------+-------+--------------------+-------------+--------------------+
|     NAME|SERIALNO|       C|     13|Housing unit/GQ p...|         NULL|                NULL|
|      VAL|SERIALNO|       C|     13|       2022GQ0000001|2022GQ9999999|GQ Unique identifier|
|      VAL|SERIALNO|       C|     13|       2022HU0000001|2022HU9999999|HU Unique identifier|
|     NAME|DIVISION|       C|      1|Division code bas...|         NULL|                NULL|
|      VAL|DIVISION|       C|      1|                   0|            0|         Puerto Rico|
|      VAL|DIVISION|       C|      1|                   1|            1|New England (Nort...|
|      VAL|DIVISION|       C|      1|                   2|            2|Middle Atlantic (...|
|      VAL|DIVISION|       C|      1|                   3|  

In [6]:
# register the data dictionary as a table so we can use SQL!
dd.createOrReplaceTempView("dd")

In [3]:
%%sparksql

SELECT field, ARRAY_AGG(code) AS codes, COUNT(*) AS n_codes FROM dd WHERE heirarchy='VAL' GROUP BY field

UsageError: Cell magic `%%sparksql` not found.


In [167]:
%%sql
-- 'b%' marks NULL values in the data with some semantic meaning. make sure the sql is right
SELECT Code, CASE Code LIKE 'b%' WHEN True THEN NULL ELSE Code END mc FROM dd WHERE Code LIKE 'b%'

Code,mc
b,None
b,None
b,None
b,None
bb,None
bb,None
b,None
b,None
bbbb,None
b,None


In [373]:
# let's use our queries now, and add a cast to integer (since everything SHOULD be int!). 
# make NULL -1 for our business logic
# subtract 1 from count because the field itself is in the data dictionary
codes_df = spark.sql("""
SELECT field, ARRAY_AGG(code) AS codes, COUNT(*) -1 AS n_codes 
FROM (
    SELECT field, cASE Code LIKE 'b%' WHEN True THEN -1 ELSE CAST (code AS int) END code FROM dd 
) GROUP BY field
""")
codes_df.where("field = 'COW'").show() # example of value with a nan

+-----+--------------------+-------+
|field|               codes|n_codes|
+-----+--------------------+-------+
|  COW|[-1, 1, 2, 3, 4, ...|     10|
+-----+--------------------+-------+



In [168]:
# sanity check. let's go to pandas because it's easier!
valid_df = codes_df.toPandas()
print(c_df)
error_idx = [ i for i, row in enumerate(c_df["codes"]) for x in row if not (x >=0 or x == -1)]
err_df = c_df.iloc[error_idx] # print the rows that dont match our assumption
err_df

          field                                              codes  n_codes
0    ACCESSINET                                      [-1, 1, 2, 3]        4
1           ACR                                      [-1, 1, 2, 3]        4
2        ADJHSG                                          [1000000]        1
3        ADJINC                                 [1042311, 1042311]        3
4          AGEP                                            [0, 99]        2
..          ...                                                ...      ...
516        WKWN                                           [-1, 52]        2
517    WORKSTAT  [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...       16
518         WRK                                         [-1, 1, 2]        3
519        YOEP  [-1, 1934, 1935, 1939, 1943, 1945, 1946, 1947,...       83
520       YRBLT  [-1, 1939, 1940, 1950, 1960, 1970, 1980, 1990,...       13

[521 rows x 3 columns]


,field,codes,n_codes
92,FINCP,"[-1, 0, -59999, -1, 9999999]",5
196,HINCP,"[-1, 0, -59999, -1, 9999999]",5
213,INTP,"[-1, 0, -4, 999999]",4
272,PERNP,"[-1, 0, -10000, -1, 1999998]",5
273,PINCP,"[-1, 0, -19998, -1, 4209995]",5
398,SEMP,"[-1, 0, -4, 999999]",4


In [169]:
# looks like our assumption was wrong 
# and there are some negative codes, plus a few cases where -1 is used as a legitimate value.
# well, let's check if we actually use any of these! 
import metadata
cols = [ md.column for md in metadata.METADATA_LIST ]
# oops, we are using some...well at least we could easily check in pandas :) 
[ (err_col, err_col in cols) for err_col in err_df["field"] ]

[('FINCP', False),
 ('HINCP', False),
 ('INTP', False),
 ('PERNP', True),
 ('PINCP', True),
 ('SEMP', False)]

In [171]:
spark.sql("SELECT CAST('bbb' AS int)").show() # test cast NULL works

+----------------+
|CAST(bbb AS INT)|
+----------------+
|            NULL|
+----------------+



In [374]:
# let's try again but we will just handle the nulls explicitly. no more -1. call it null_str so its not confusing
# *** happy platform team noises not having to maintain the -1 ***
# add logic to see the category (number N or categorical C) 
codes_df = spark.sql("""
SELECT *, (n_codes_raw - has_null) n_codes
FROM (
    SELECT field, ANY_VALUE(category) category, ARRAY_AGG(code) AS codes, ARRAY_AGG(text) AS texts, MAX(CASE WHEN code = "null_str" THEN 1 ELSE 0 END) has_null, COUNT(*) AS n_codes_raw
    FROM (
        SELECT field, category, text, (CASE WHEN code LIKE 'b%' THEN 'null_str' ELSE code END) code FROM dd WHERE heirarchy = 'VAL'
    ) 
    GROUP BY field
)
""")
codes_df.where("has_null IS TRUE").show(5) # check non columns to make sure processing was sane
codes_df.createOrReplaceTempView("codes_df")

+----------+--------+--------------------+--------------------+--------+-----------+-------+
|     field|category|               codes|               texts|has_null|n_codes_raw|n_codes|
+----------+--------+--------------------+--------------------+--------+-----------+-------+
|ACCESSINET|       C| [null_str, 1, 2, 3]|[N/A (GQ/vacant),...|       1|          4|      3|
|       ACR|       C| [null_str, 1, 2, 3]|[N/A (GQ/not a on...|       1|          4|      3|
|       AGS|       C|[null_str, 1, 2, ...|[N/A (GQ/vacant/n...|       1|          7|      6|
|      BATH|       C|    [null_str, 1, 2]| [N/A (GQ), Yes, No]|       1|          3|      2|
|      BDSP|       N|      [null_str, 99]|[N/A (GQ), 0 to 9...|       1|          2|      1|
+----------+--------+--------------------+--------------------+--------+-----------+-------+
only showing top 5 rows



In [239]:
%%sql
-- separate metadata into types of categories. lets see how our query does.
-- note that this was built up incrementally, adding the bounds at the end.

SELECT field, has_null, CASE category WHEN 'N' THEN 'number' WHEN 'C' THEN
    CASE WHEN n_codes <= 2 THEN 'binary' WHEN n_codes <= 10 THEN 'small categorical' ELSE 'large categorical' END 
ELSE 'ERROR' END cardinality, codes, codes[has_null] lower_bound, codes[n_codes-1+has_null] upper_bound
FROM codes_df

field,has_null,cardinality,codes,lower_bound,upper_bound
ACCESSINET,1,small categorical,"['null_str', '1', '2', '3']",1,3
ACR,1,small categorical,"['null_str', '1', '2', '3']",1,3
ADJHSG,0,binary,['1000000'],1000000,1000000
ADJINC,0,binary,"['1042311', '1042311']",1042311,1042311
AGEP,0,number,"['0', '99']",0,99
AGS,1,small categorical,"['null_str', '1', '2', '3', '4', '5', '6']",1,6
ANC,0,small categorical,"['1', '2', '3', '4']",1,4
ANC1P,0,large categorical,"['1', '3', '5', '8', '9', '11', '12', '20', '21', '22', '24', '26', '32', '40', '46', '49', '50', '51', '68', '77', '78', '82', '84', '87', '88', '89', '91', '94', '97', '98', '99', '100', '102', '103', '109', '111', '112', '114', '115', '122', '124', '125', '128', '129', '130', '131', '142', '144', '146', '148', '152', '153', '154', '168', '169', '170', '171', '176', '177', '178', '181', '183', '185', '187', '190', '194', '195', '200', '210', '211', '212', '213', '215', '218', '219', '221', '222', '223', '224', '225', '226', '227', '231', '232', '233', '234', '235', '236', '237', '238', '239', '249', '250', '251', '252', '261', '271', '275', '290', '291', '295', '300', '301', '302', '308', '310', '314', '322', '325', '329', '330', '331', '335', '336', '359', '360', '370', '400', '402', '404', '406', '411', '416', '417', '419', '421', '425', '427', '429', '431', '434', '435', '442', '465', '483', '484', '490', '495', '496', '499', '508', '510', '515', '522', '523', '527', '529', '530', '534', '541', '553', '564', '566', '568', '570', '576', '586', '587', '588', '593', '598', '599', '600', '603', '607', '609', '615', '618', '620', '650', '680', '690', '700', '703', '706', '707', '712', '714', '716', '720', '730', '740', '748', '750', '765', '768', '770', '776', '782', '785', '795', '799', '800', '803', '808', '811', '814', '815', '820', '821', '822', '825', '841', '850', '899', '900', '901', '902', '903', '904', '907', '913', '914', '917', '918', '919', '920', '922', '924', '925', '927', '929', '931', '935', '937', '939', '940', '983', '994', '995', '996', '997', '998', '999']",1,999
ANC2P,0,large categorical,"['1', '3', '5', '8', '9', '11', '12', '20', '21', '22', '24', '26', '32', '40', '46', '49', '50', '51', '68', '77', '78', '82', '84', '87', '88', '89', '91', '94', '97', '98', '99', '100', '102', '103', '109', '111', '112', '114', '115', '122', '124', '125', '128', '129', '130', '131', '142', '144', '146', '148', '152', '153', '154', '168', '169', '170', '171', '176', '177', '178', '181', '183', '185', '187', '190', '194', '195', '200', '210', '211', '212', '213', '215', '218', '219', '221', '222', '223', '224', '225', '226', '227', '231', '232', '233', '234', '235', '236', '237', '238', '239', '249', '250', '251', '252', '261', '271', '275', '290', '291', '295', '300', '301', '302', '308', '310', '314', '322', '325', '329', '330', '331', '335', '336', '359', '360', '370', '400', '402', '404', '406', '411', '416', '417', '419', '421', '425', '427', '429', '431', '434', '435', '442', '465', '483', '484', '490', '495', '496', '499', '508', '510', '515', '522', '523', '527', '529', '530', '534', '541', '553', '564', '566', '568', '570', '576', '586', '587', '588', '593', '598', '599', '600', '603', '607', '609', '615', '618', '620', '650', '680', '690', '700', '703', '706', '707', '712', '714', '716', '720', '730', '740', '748', '750', '765', '768', '770', '776', '782', '785', '795', '799', '800', '803', '808', '811', '814', '815', '820', '821', '822', '825', '841', '850', '899', '900', '901', '902', '903', '904', '907', '913', '914', '917', '918', '919', '920', '922', '924', '925', '927', '929', '931', '935', '937', '939', '940', '983', '994', '995', '996', '997', '998', '999']",1,999
BATH,1,binary,"['null_str', '1', '2']",1,2


In [375]:
# okay seems good. a bit hacky though, so this is the type of change we want to
# check in a staging area before committing. 

aug_codes_df = spark.sql("""
SELECT *, CASE category WHEN 'N' THEN 'number' WHEN 'C' THEN
    CASE WHEN n_codes <= 2 THEN 'binary' WHEN n_codes <= 10 THEN 'small categorical' ELSE 'large categorical' END 
ELSE 'ERROR' END cardinality, CAST(codes[has_null] AS int) lower_bound, CAST(codes[n_codes-1+has_null] AS int) upper_bound
FROM codes_df
""")
aug_codes_df.show(10)

+----------+--------+--------------------+--------------------+--------+-----------+-------+-----------------+-----------+-----------+
|     field|category|               codes|               texts|has_null|n_codes_raw|n_codes|      cardinality|lower_bound|upper_bound|
+----------+--------+--------------------+--------------------+--------+-----------+-------+-----------------+-----------+-----------+
|ACCESSINET|       C| [null_str, 1, 2, 3]|[N/A (GQ/vacant),...|       1|          4|      3|small categorical|          1|          3|
|       ACR|       C| [null_str, 1, 2, 3]|[N/A (GQ/not a on...|       1|          4|      3|small categorical|          1|          3|
|    ADJHSG|       C|           [1000000]|[2022 factor (1.0...|       0|          1|      1|           binary|    1000000|    1000000|
|    ADJINC|       C|  [1042311, 1042311]|[2022 factor (1.0...|       0|          2|      2|           binary|    1042311|    1042311|
|      AGEP|       N|             [0, 99]|[Under 1 year

In [376]:
# insert df in staging
aug_codes_df.writeTo(f"{metadata.TABLESPACE}.stage.dd").createOrReplace()

In [377]:
%%sql
-- check we got same number of description and codes (correct null handling)
SELECT * FROM nessie.stage.dd WHERE cardinality(codes) != cardinality(texts) LIMIT 10

field,category,codes,texts,has_null,n_codes_raw,n_codes,cardinality,lower_bound,upper_bound


In [378]:
%%sql
-- check we did not fail to insert into the table!
SELECT COUNT(1) FROM nessie.stage.dd WHERE cardinality(codes) == cardinality(texts)

count(1)
521


In [379]:
%%sql
-- some additional sanity checks. not-zero is okay but maybe worth checking

SELECT ARRAY_AGG(field) FILTER (WHERE cardinality(codes) = 0) no_examples, -- this is an error
COUNT(field) FILTER (WHERE lower_bound IS NULL) null_lb, 
COUNT(field) FILTER (WHERE upper_bound IS NULL) null_ub,
COUNT(has_null) FILTER (WHERE has_null != 0 AND has_null != 1) wrong_has_null -- error
FROM nessie.stage.dd

no_examples,null_lb,null_ub,wrong_has_null
[],3,3,0


In [306]:
# okay i'm satisfied. let's put this table in bronze finally! data alignment complete!
spark.sql("USE REFERENCE main IN nessie")

DataFrame[refType: string, name: string, hash: string]

In [380]:
spark.sql("SELECT * FROM nessie.stage.dd").writeTo(f"{metadata.TABLESPACE}.bronze.dd").createOrReplace()
spark.sql("SELECT * FROM nessie.bronze.dd").show(20) # insertion worked!

+----------+--------+--------------------+--------------------+--------+-----------+-------+-----------------+-----------+-----------+
|     field|category|               codes|               texts|has_null|n_codes_raw|n_codes|      cardinality|lower_bound|upper_bound|
+----------+--------+--------------------+--------------------+--------+-----------+-------+-----------------+-----------+-----------+
|ACCESSINET|       C| [null_str, 1, 2, 3]|[N/A (GQ/vacant),...|       1|          4|      3|small categorical|          1|          3|
|       ACR|       C| [null_str, 1, 2, 3]|[N/A (GQ/not a on...|       1|          4|      3|small categorical|          1|          3|
|    ADJHSG|       C|           [1000000]|[2022 factor (1.0...|       0|          1|      1|           binary|    1000000|    1000000|
|    ADJINC|       C|  [1042311, 1042311]|[2022 factor (1.0...|       0|          2|      2|           binary|    1042311|    1042311|
|      AGEP|       N|             [0, 99]|[Under 1 year

In [314]:
%%sql
-- oops. product wants us to include a name description for the column.
-- let's add that in a safeish way. first by checking the new sql
SELECT sdd.field, sdd.category, name_text FROM nessie.stage.dd AS sdd LEFT  JOIN (
    SELECT field, name_text FROM dd WHERE heirarchy="NAME"
) AS dd_name
ON sdd.field = dd_name.field LIMIT 10

field,category,name_text
ACCESSINET,C,Access to the Internet
ACR,C,Lot size
ADJHSG,C,Adjustment factor for housing dollar amounts (6 implied decimal places)
ADJINC,C,Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
ADJINC,C,Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
AGEP,N,Age
AGS,C,"Sales of Agriculture Products (yearly sales, no adjustment factor is applied)"
ANC,C,Ancestry recode
ANC1P,C,Recoded Detailed Ancestry - first entry
ANC2P,C,Recoded Detailed Ancestry - second entry


In [8]:
dd_bronze = spark.sql("""
SELECT sdd.*, name_text FROM nessie.stage.dd AS sdd INNER JOIN (
    SELECT DISTINCT field, name_text FROM dd WHERE heirarchy="NAME"
) AS dd_name
ON sdd.field = dd_name.field ORDER BY sdd.field
""")
dd_bronze.show(20)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `dd` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 42;
'Sort ['sdd.field ASC NULLS FIRST], true
+- 'Project [sdd.*, 'name_text]
   +- 'Join Inner, ('sdd.field = 'dd_name.field)
      :- SubqueryAlias sdd
      :  +- SubqueryAlias nessie.stage.dd
      :     +- RelationV2[field#0, category#1, codes#2, texts#3, has_null#4, n_codes_raw#5L, n_codes#6L, cardinality#7, lower_bound#8, upper_bound#9] nessie.stage.dd nessie.stage.dd
      +- 'SubqueryAlias dd_name
         +- 'Distinct
            +- 'Project ['field, 'name_text]
               +- 'Filter ('heirarchy = NAME)
                  +- 'UnresolvedRelation [dd], [], false


In [358]:
spark.sql("CREATE BRANCH IF NOT EXISTS data_dictionary2")
spark.sql("USE REFERENCE data_dictionary2 IN nessie")

DataFrame[refType: string, name: string, hash: string]

In [359]:
%%sql
LIST REFERENCES IN nessie

refType,name,hash
Branch,data_dictionary,c074dc1450978801fd155ead659190f6498386aa5c3251a5ccfda59085095c86
Branch,data_dictionary2,4dee9588d3c2077e539e827fe16c92a764df0e7270546a808e609b0cc73c9c00
Branch,main,4dee9588d3c2077e539e827fe16c92a764df0e7270546a808e609b0cc73c9c00


In [318]:
%%sql
SHOW REFERENCE IN nessie

refType,name,hash
Branch,data_dictionary,0df07897c9b8bc6b2314f6bc70b1e02aa4cb2bf888c3188835596d8e0ffdafb8


In [387]:
# okay, i'm convinced we actually made a branch. let's insert and take one last check.
dd_bronze.writeTo(f"{metadata.TABLESPACE}.bronze.ddm").createOrReplace()

In [388]:
# we can also check that the original table is still there. 
# we did not break anything (yet)
print("see branch info...")
spark.sql("USE REFERENCE data_dictionary2 IN nessie")
spark.sql("DESCRIBE TABLE nessie.bronze.ddm").show()
spark.sql("SELECT COUNT(1) FROM nessie.bronze.ddm").show()
print("see main info...")
spark.sql("USE REFERENCE main IN nessie")
spark.sql("DESCRIBE TABLE nessie.bronze.ddm").show()
spark.sql("SELECT COUNT(1) FROM nessie.bronze.ddm").show()

see branch info...
+-----------+-------------+-------+
|   col_name|    data_type|comment|
+-----------+-------------+-------+
|      field|       string|   NULL|
|   category|       string|   NULL|
|      codes|array<string>|   NULL|
|      texts|array<string>|   NULL|
|   has_null|          int|   NULL|
|n_codes_raw|       bigint|   NULL|
|    n_codes|       bigint|   NULL|
|cardinality|       string|   NULL|
|lower_bound|          int|   NULL|
|upper_bound|          int|   NULL|
|  name_text|       string|   NULL|
+-----------+-------------+-------+

+--------+
|count(1)|
+--------+
|     527|
+--------+

see main info...
+-----------+-------------+-------+
|   col_name|    data_type|comment|
+-----------+-------------+-------+
|      field|       string|   NULL|
|   category|       string|   NULL|
|      codes|array<string>|   NULL|
|      texts|array<string>|   NULL|
|   has_null|          int|   NULL|
|n_codes_raw|       bigint|   NULL|
|    n_codes|       bigint|   NULL|
|cardin

In [389]:
# uh oh. the counts are different :eyes: 
# okay this broke figure it out tomm.)
spark.sql("DROP TABLE IF EXISTS nessie.bronze.dd")
spark.sql("MERGE BRANCH data_dictionary2 INTO main IN nessie")
spark.sql("USE REFERENCE main IN nessie")
spark.sql("DESCRIBE TABLE nessie.bronze.ddm").show()

+-----------+-------------+-------+
|   col_name|    data_type|comment|
+-----------+-------------+-------+
|      field|       string|   NULL|
|   category|       string|   NULL|
|      codes|array<string>|   NULL|
|      texts|array<string>|   NULL|
|   has_null|          int|   NULL|
|n_codes_raw|       bigint|   NULL|
|    n_codes|       bigint|   NULL|
|cardinality|       string|   NULL|
|lower_bound|          int|   NULL|
|upper_bound|          int|   NULL|
|  name_text|       string|   NULL|
+-----------+-------------+-------+



In [6]:
%%sparksql --output grid

SELECT * FROM nessie.bronze.ddm LIMIT 10

UsageError: Cell magic `%%sparksql` not found.
